In [1]:
!pip install transformers accelerate evaluate datasets peft -q
!pip install diffusers transformers accelerate
!pip install pyngrok
!pip install flask-ngrok
!pip install -q diffusers==0.14.0 transformers xformers git+https://github.com/huggingface/accelerate.git
!pip install -q opencv-contrib-python
!pip install -q controlnet_aux
from diffusers import StableDiffusionControlNetPipeline
from diffusers.utils import load_image
import cv2
from PIL import Image
import numpy as np
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel
import torch

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.1.0+cu118)
    Python  3.10.13 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


In [2]:
from huggingface_hub import notebook_login
from transformers import AutoImageProcessor, UperNetForSemanticSegmentation, AutoModelForCausalLM, AutoProcessor
from PIL import Image
import numpy as np
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from diffusers.utils import load_image
from peft import LoraConfig, get_peft_model
from peft import PeftModel
from peft import PeftConfig
import json
from huggingface_hub import cached_download, hf_hub_url
from datasets import load_dataset
import requests
from transformers import AutoImageProcessor
from transformers import AutoModelForSemanticSegmentation, TrainingArguments, Trainer
from datasets import concatenate_datasets
from datasets import Dataset, load_dataset

notebook_login('gychoi')

In [3]:
ds = load_dataset("EduardoPacheco/FoodSeg103",split='train[:1000]')
# ds = concatenate_datasets([ds['train'],ds['validation']])

In [4]:
# ind_list = []
# for ind,i in enumerate(ds):
#   if (0!=sum(np.unique(np.array(i['label']))==25)):
#     ind_list.append(ind)

In [5]:
# selected_indexes = ind_list
selected_indexes = [22,22]
selected_data = {feature: [ds[i][feature] for i in selected_indexes] for feature in ds.features}
new_dataset = Dataset.from_dict(selected_data)
print(new_dataset)
indices_to_extract = list(np.unique(np.array([np.unique(np.array(i['label'])) for i in new_dataset])))


Dataset({
    features: ['image', 'label'],
    num_rows: 2
})


In [6]:
new_dataset = new_dataset.train_test_split(test_size=0.01)
train_ds = new_dataset["train"]
test_ds = new_dataset["test"]

In [7]:
indices_to_extract = [i for i in range(46)]

In [8]:
file_path = './id2label.json'
with open(file_path, 'r') as file:
    id2label = json.load(file)
extracted_dict = {
    index: id2label[str(index)] if index != 0 else id2label[str(index)]
    for index in indices_to_extract
}
id2label = extracted_dict
# id2label = {0: 'background', 13: 'juice', 24: 'apple', 28: 'banana', 43: 'orange',255: 'background2'}
id2label = {int(k): v for k, v in id2label.items()}
label2id = {v: k for k, v in id2label.items()}

num_labels = len(id2label)

In [9]:
id2label

{0: 'background',
 1: 'candy',
 2: 'egg tart',
 3: 'french fries',
 4: 'chocolate',
 5: 'biscuit',
 6: 'popcorn',
 7: 'pudding',
 8: 'ice cream',
 9: 'cheese butter',
 10: 'cake',
 11: 'wine',
 12: 'milkshake',
 13: 'coffee',
 14: 'juice',
 15: 'milk',
 16: 'tea',
 17: 'almond',
 18: 'red beans',
 19: 'cashew',
 20: 'dried cranberries',
 21: 'soy',
 22: 'walnut',
 23: 'peanut',
 24: 'egg',
 25: 'apple',
 26: 'date',
 27: 'apricot',
 28: 'avocado',
 29: 'banana',
 30: 'strawberry',
 31: 'cherry',
 32: 'blueberry',
 33: 'raspberry',
 34: 'mango',
 35: 'olives',
 36: 'peach',
 37: 'lemon',
 38: 'pear',
 39: 'fig',
 40: 'pineapple',
 41: 'grape',
 42: 'kiwi',
 43: 'melon',
 44: 'orange',
 45: 'watermelon'}

In [10]:
checkpoint = "nvidia/segformer-b2-finetuned-cityscapes-1024-1024"
image_processor = AutoImageProcessor.from_pretrained(checkpoint, do_reduce_labels=True)

model = AutoModelForSemanticSegmentation.from_pretrained(
    checkpoint, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True
)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/usr/local/lib/python3.10/dist-packages/transformers/models/segformer/image_processing_segformer.py:101: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b2-finetuned-cityscapes-1024-1024 and are newly initialized because the shapes did not match:
- decode_head.classifier.weight: found shape torch.Size([19, 768, 1, 1]) in the checkpoint and torch.Size([46, 768, 1, 1]) in the model instantiated
- decode_head.classifier.bias: found shape torch.Size([19]) in the checkpoint and torch.Size([46]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for

In [11]:
from torchvision.transforms import ColorJitter

jitter = ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1)

In [12]:
import numpy as np

def handle_grayscale_image(image):
    np_image = np.array(image)
    if np_image.ndim == 2:
        tiled_image = np.tile(np.expand_dims(np_image, -1), 3)
        return Image.fromarray(tiled_image)
    else:
        return Image.fromarray(np_image)

In [13]:
from PIL import Image


def train_transforms(example_batch):
    images = [jitter(handle_grayscale_image(x)) for x in example_batch["image"]]
    labels = [x for x in example_batch["label"]]
    inputs = image_processor(images, labels)
    return inputs


def val_transforms(example_batch):
    images = [handle_grayscale_image(x) for x in example_batch["image"]]
    labels = [x for x in example_batch["label"]]
    inputs = image_processor(images, labels)
    return inputs

In [14]:
train_ds.set_transform(train_transforms)
test_ds.set_transform(val_transforms)

In [15]:
import torch
from torch import nn
import evaluate

metric = evaluate.load("mean_iou")


def compute_metrics(eval_pred):
    with torch.no_grad():
        logits, labels = eval_pred
        logits_tensor = torch.from_numpy(logits)
        logits_tensor = nn.functional.interpolate(
            logits_tensor,
            size=labels.shape[-2:],
            mode="bilinear",
            align_corners=False,
        ).argmax(dim=1)

        pred_labels = logits_tensor.detach().cpu().numpy()
        # currently using _compute instead of compute
        # see this issue for more info: https://github.com/huggingface/evaluate/pull/328#issuecomment-1286866576
        metrics = metric._compute(
            predictions=pred_labels,
            references=labels,
            num_labels=len(id2label),
            ignore_index=0,
            reduce_labels=image_processor.do_reduce_labels,
        )

        per_category_accuracy = metrics.pop("per_category_accuracy").tolist()
        per_category_iou = metrics.pop("per_category_iou").tolist()

        metrics.update({f"accuracy_{id2label[i]}": v for i, v in enumerate(per_category_accuracy)})
        metrics.update({f"iou_{id2label[i]}": v for i, v in enumerate(per_category_iou)})

        return metrics

In [16]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [17]:
# from transformers import AutoModelForSemanticSegmentation, TrainingArguments, Trainer
# from transformers import AutoImageProcessor

checkpoint = "nvidia/segformer-b2-finetuned-cityscapes-1024-1024"
image_processor = AutoImageProcessor.from_pretrained(checkpoint, do_reduce_labels=True)

model = AutoModelForSemanticSegmentation.from_pretrained(
    checkpoint, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True
)
print_trainable_parameters(model)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/usr/local/lib/python3.10/dist-packages/transformers/models/segformer/image_processing_segformer.py:101: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b2-finetuned-cityscapes-1024-1024 and are newly initialized because the shapes did not match:
- decode_head.classifier.weight: found shape torch.Size([19, 768, 1, 1]) in the checkpoint and torch.Size([46, 768, 1, 1]) in the model instantiated
- decode_head.classifier.bias: found shape torch.Size([19]) in the checkpoint and torch.Size([46]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for

trainable params: 27381998 || all params: 27381998 || trainable%: 100.00


In [18]:
config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=["query", "value"],
    lora_dropout=0.1,
    bias="lora_only",
    modules_to_save=["decode_head"],
)
lora_model = get_peft_model(model, config)
print_trainable_parameters(lora_model)

trainable params: 3726510 || all params: 31100188 || trainable%: 11.98


In [19]:
# for name, param in lora_model.named_parameters():
#     if param.requires_grad:
#         print(name, param.shape)

In [20]:
model_name = checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-lora",
    learning_rate=5e-4,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=2,
    save_total_limit=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=5,
    remove_unused_columns=False,
    # push_to_hub=True,
    label_names=["labels"],
)

In [21]:
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Mean Iou,Mean Accuracy,Overall Accuracy,Accuracy Background,Accuracy Candy,Accuracy Egg tart,Accuracy French fries,Accuracy Chocolate,Accuracy Biscuit,Accuracy Popcorn,Accuracy Pudding,Accuracy Ice cream,Accuracy Cheese butter,Accuracy Cake,Accuracy Wine,Accuracy Milkshake,Accuracy Coffee,Accuracy Juice,Accuracy Milk,Accuracy Tea,Accuracy Almond,Accuracy Red beans,Accuracy Cashew,Accuracy Dried cranberries,Accuracy Soy,Accuracy Walnut,Accuracy Peanut,Accuracy Egg,Accuracy Apple,Accuracy Date,Accuracy Apricot,Accuracy Avocado,Accuracy Banana,Accuracy Strawberry,Accuracy Cherry,Accuracy Blueberry,Accuracy Raspberry,Accuracy Mango,Accuracy Olives,Accuracy Peach,Accuracy Lemon,Accuracy Pear,Accuracy Fig,Accuracy Pineapple,Accuracy Grape,Accuracy Kiwi,Accuracy Melon,Accuracy Orange,Accuracy Watermelon,Iou Background,Iou Candy,Iou Egg tart,Iou French fries,Iou Chocolate,Iou Biscuit,Iou Popcorn,Iou Pudding,Iou Ice cream,Iou Cheese butter,Iou Cake,Iou Wine,Iou Milkshake,Iou Coffee,Iou Juice,Iou Milk,Iou Tea,Iou Almond,Iou Red beans,Iou Cashew,Iou Dried cranberries,Iou Soy,Iou Walnut,Iou Peanut,Iou Egg,Iou Apple,Iou Date,Iou Apricot,Iou Avocado,Iou Banana,Iou Strawberry,Iou Cherry,Iou Blueberry,Iou Raspberry,Iou Mango,Iou Olives,Iou Peach,Iou Lemon,Iou Pear,Iou Fig,Iou Pineapple,Iou Grape,Iou Kiwi,Iou Melon,Iou Orange,Iou Watermelon
1,No log,3.508463,0.000000,0.000000,0.000000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.000000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.000000,nan,nan,nan,0.000000,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,0.000000,nan,nan,nan,0.000000,0.000000,0.000000,0.000000,0.000000,nan,0.000000,nan,0.000000,0.000000,nan,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,nan,0.000000,0.000000,0.000000,0.000000,0.000000,nan,0.000000,0.000000,0.000000,0.000000,nan,0.000000,0.000000,0.000000,0.000000


/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:259: RuntimeWarning: invalid value encountered in divide
  iou = total_area_intersect / total_area_union
/root/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--mean_iou/08bc20f4f895f3caf75fb9e3fada1404bded3c3265243d05327cbb3b9326ffe9/mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label


TrainOutput(global_step=1, training_loss=3.788627862930298, metrics={'train_runtime': 2.2572, 'train_samples_per_second': 0.443, 'train_steps_per_second': 0.443, 'total_flos': 146749098295296.0, 'train_loss': 3.788627862930298, 'epoch': 1.0})

In [22]:
model_id = "segformer-food1"
lora_model.save_pretrained(model_id)

In [23]:
config = PeftConfig.from_pretrained(model_id)
model = AutoModelForSemanticSegmentation.from_pretrained(
    checkpoint, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True
)

inference_model = PeftModel.from_pretrained(model, model_id)

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/segformer-b2-finetuned-cityscapes-1024-1024 and are newly initialized because the shapes did not match:
- decode_head.classifier.weight: found shape torch.Size([19, 768, 1, 1]) in the checkpoint and torch.Size([46, 768, 1, 1]) in the model instantiated
- decode_head.classifier.bias: found shape torch.Size([19]) in the checkpoint and torch.Size([46]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
checkpoint_text_cap = "microsoft/git-large-textcaps"

processor_cap = AutoProcessor.from_pretrained(checkpoint_text_cap)
model_cap = AutoModelForCausalLM.from_pretrained(checkpoint_text_cap)

In [30]:
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-seg", torch_dtype=torch.float16
)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, safety_checker=None, torch_dtype=torch.float16
)
pipe.enable_model_cpu_offload()

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_controlnet.StableDiffusionControlNetPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [31]:
def ade_palette():
    """ADE20K palette that maps each class to RGB values."""
    return [[120, 120, 120], [180, 120, 120], [6, 230, 230], [80, 50, 50],
            [4, 200, 3], [120, 120, 80], [140, 140, 140], [204, 5, 255],
            [230, 230, 230], [4, 250, 7], [224, 5, 255], [235, 255, 7],
            [150, 5, 61], [120, 120, 70], [8, 255, 51], [255, 6, 82],
            [143, 255, 140], [204, 255, 4], [255, 51, 7], [204, 70, 3],
            [0, 102, 200], [61, 230, 250], [255, 6, 51], [11, 102, 255],
            [255, 7, 71], [255, 9, 224], [9, 7, 230], [220, 220, 220],
            [255, 9, 92], [112, 9, 255], [8, 255, 214], [7, 255, 224],
            [255, 184, 6], [10, 255, 71], [255, 41, 10], [7, 255, 255],
            [224, 255, 8], [102, 8, 255], [255, 61, 6], [255, 194, 7],
            [255, 122, 8], [0, 255, 20], [255, 8, 41], [255, 5, 153],
            [6, 51, 255], [235, 12, 255], [160, 150, 20], [0, 163, 255],
            [140, 140, 140], [250, 10, 15], [20, 255, 0], [31, 255, 0],
            [255, 31, 0], [255, 224, 0], [153, 255, 0], [0, 0, 255],
            [255, 71, 0], [0, 235, 255], [0, 173, 255], [31, 0, 255],
            [11, 200, 200], [255, 82, 0], [0, 255, 245], [0, 61, 255],
            [0, 255, 112], [0, 255, 133], [255, 0, 0], [255, 163, 0],
            [255, 102, 0], [194, 255, 0], [0, 143, 255], [51, 255, 0],
            [0, 82, 255], [0, 255, 41], [0, 255, 173], [10, 0, 255],
            [173, 255, 0], [0, 255, 153], [255, 92, 0], [255, 0, 255],
            [255, 0, 245], [255, 0, 102], [255, 173, 0], [255, 0, 20],
            [255, 184, 184], [0, 31, 255], [0, 255, 61], [0, 71, 255],
            [255, 0, 204], [0, 255, 194], [0, 255, 82], [0, 10, 255],
            [0, 112, 255], [51, 0, 255], [0, 194, 255], [0, 122, 255],
            [0, 255, 163], [255, 153, 0], [0, 255, 10], [255, 112, 0],
            [143, 255, 0], [82, 0, 255], [163, 255, 0], [255, 235, 0],
            [8, 184, 170], [133, 0, 255], [0, 255, 92], [184, 0, 255],
            [255, 0, 31], [0, 184, 255], [0, 214, 255], [255, 0, 112],
            [92, 255, 0], [0, 224, 255], [112, 224, 255], [70, 184, 160],
            [163, 0, 255], [153, 0, 255], [71, 255, 0], [255, 0, 163],
            [255, 204, 0], [255, 0, 143], [0, 255, 235], [133, 255, 0],
            [255, 0, 235], [245, 0, 255], [255, 0, 122], [255, 245, 0],
            [10, 190, 212], [214, 255, 0], [0, 204, 255], [20, 0, 255],
            [255, 255, 0], [0, 153, 255], [0, 41, 255], [0, 255, 204],
            [41, 0, 255], [41, 255, 0], [173, 0, 255], [0, 245, 255],
            [71, 0, 255], [122, 0, 255], [0, 255, 184], [0, 92, 255],
            [184, 255, 0], [0, 133, 255], [255, 214, 0], [25, 194, 194],
            [102, 255, 0], [92, 0, 255]]

palette = np.array(ade_palette())

In [ ]:
import matplotlib.pyplot as plt

image_path = 'ex1.png'
image = Image.open(image_path)
inputs = processor_cap(images=image, return_tensors="pt")
pixel_values = inputs.pixel_values
generated_ids = model_cap.generate(pixel_values=pixel_values, max_length=300)
prompt = processor_cap.batch_decode(generated_ids, skip_special_tokens=True)[0]

new_size = (512, 512)
resized_image = image.resize(new_size)
resized_image.show()
encoding = image_processor(image.convert("RGB"), return_tensors="pt")
with torch.no_grad():
    outputs = inference_model(pixel_values=encoding.pixel_values)
    logits = outputs.logits

upsampled_logits = nn.functional.interpolate(
    logits,
    size=image.size[::-1],
    mode="bilinear",
    align_corners=False,
)

pred_seg = upsampled_logits.argmax(dim=1)[0]

color_seg = np.zeros((pred_seg.shape[0], pred_seg.shape[1], 3), dtype=np.uint8)
palette = np.array(ade_palette())

for label, color in enumerate(palette):
    color_seg[pred_seg == label, :] = color
color_seg = color_seg[..., ::-1]  # convert to BGR

img = color_seg * 1  # plot the image with the segmentation map
img = img.astype(np.uint8)

plt.figure(figsize=(15, 10))
plt.imshow(img)
plt.show()
image = Image.fromarray(color_seg)
image = Image.fromarray(img)

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
generated_image = pipe(prompt+" , best quality, extremely detailed",image, num_inference_steps=20).images[0]
generated_image

In [ ]:
from flask import Flask, request, jsonify
from PIL import Image
import io
from pyngrok import ngrok
from io import BytesIO
import base64

app = Flask(__name__)

@app.route('/caption', methods=['POST'])
def caption_image():
    print("Success reached")
    if 'file' not in request.files or 'description' not in request.form:
        print("problem 1")
        return jsonify({'error': 'Missing file or description'})
    print("problem 1 passed")
    file = request.files['file']
    print("file extracted")
    description = request.form['description']
    print("description extracted")
    # music=request.form['music']
    print("file and description gotten")
    print("file",file)
    print("description",description)
    if file.filename == '' or description == '':
        print("problem 2")
        return jsonify({'error': 'Please select a file and provide a description'})

    # Process the image file
    controlnet = ControlNetModel.from_pretrained(
        "lllyasviel/sd-controlnet-seg", torch_dtype=torch.float16
    )

    pipe = StableDiffusionControlNetPipeline.from_pretrained(
        "runwayml/stable-diffusion-v1-5", controlnet=controlnet, safety_checker=None, torch_dtype=torch.float16
    )
    pipe.enable_model_cpu_offload()

    image_stream = io.BytesIO()
    file.save(image_stream)
    image = Image.open(image_stream)
    inputs = processor_cap(images=image, return_tensors="pt")
    pixel_values = inputs.pixel_values
    generated_ids = model_cap.generate(pixel_values=pixel_values, max_length=300)
    prompt = processor_cap.batch_decode(generated_ids, skip_special_tokens=True)[0]

    new_size = (512, 512)
    resized_image = image.resize(new_size)
    resized_image.show()
    encoding = image_processor(image.convert("RGB"), return_tensors="pt")
    with torch.no_grad():
        outputs = inference_model(pixel_values=encoding.pixel_values)
        logits = outputs.logits

    upsampled_logits = nn.functional.interpolate(
        logits,
        size=image.size[::-1],
        mode="bilinear",
        align_corners=False,
    )

    pred_seg = upsampled_logits.argmax(dim=1)[0]

    color_seg = np.zeros((pred_seg.shape[0], pred_seg.shape[1], 3), dtype=np.uint8)
    palette = np.array(ade_palette())

    for label, color in enumerate(palette):
        color_seg[pred_seg == label, :] = color
    color_seg = color_seg[..., ::-1]  # convert to BGR

    img = color_seg * 1  # plot the image with the segmentation map
    img = img.astype(np.uint8)
    image = Image.fromarray(color_seg)
    image = Image.fromarray(img)

    pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
    generated_image = pipe(prompt+" , best quality, extremely detailed",image, num_inference_steps=20).images[0]

    caption = prompt
    # You can now send both the image and the description to the Colab notebook
    # Use a POST request to your Colab endpoint with both data
    # Example:
    # requests.post('YOUR_COLAB_ENDPOINT', files={'file': file}, data={'description': description})
    print("works")

    buffered = BytesIO()
    generated_image.save(buffered, format="PNG")
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    print("caption",caption)
    print("image",image)
    print('description',description)
    return jsonify({'caption': caption, 'image': img_str,'description': description})

if __name__ == '__main__':
    port = 5000  # Replace with your Flask app port
    public_url = ngrok.connect(addr=f'http://localhost:{port}')
    print(f"Running Flask app at: {public_url}")
    app.run(port=port)
#provide generated url . ex. https://778e-34-125-175-211.ngrok.io
#https://www.youtube.com/watch?v=nUrHgibTnkM
#try connecting colab to vm

Running Flask app at: NgrokTunnel: "https://1ef4-35-201-136-92.ngrok.io" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


Success reached
problem 1 passed
file extracted
description extracted
file and description gotten
file <FileStorage: 'ex1.png' (None)>
description banana


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_controlnet.StableDiffusionControlNetPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


  0%|          | 0/20 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [03/Dec/2023 18:48:02] "POST /caption HTTP/1.1" 200 -


works
caption a glass of fruit sits on a blue table with a banana, orange, and an apple.
image <PIL.Image.Image image mode=RGB size=568x760 at 0x7D37BE7DAC20>
description banana
